In [4]:
from pyspark.sql import Row, functions as f
from pyspark.sql.types import *
from pyspark.sql.types import StructType
import pandas as pd
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
import re
from pyspark.sql.functions import udf

In [5]:
from pyspark import SparkContext

### Importing raw weather file into spark RDD

In [94]:
raw_Weather_file = sc.textFile("/home/ragavan/Downloads/data1.csv")

In [126]:
#spliting raw file using delimiter
raw_file_split=raw_Weather_file.map(lambda x: x.split(","))
raw_file_split.take(1)

In [127]:
#creating a spark DataFrame for weather data
Weather_data =raw_file_split.toDF(['Location','PostCode','Temp_C','Dew_Point','Rel_Hum_%','Press_hPa','Altitude_m','Rain_mm','Low_Temp_C','High_Temp_C','Latitude','longtitude','Time_Stamp'])
Weather_data.show()

In [66]:
#Registering the weather dataframe to a TempTable
Weather_data.registerTempTable("Weather_data")


In [67]:
#importing raw post_code file into spark RDD
Raw_Post_code_data = sc.textFile("/home/ragavan/Downloads/postcode.csv")
#spliting raw file using delimiter
Raw_Post_code_data_split=Raw_Post_code_data.map(lambda x: x.split(","))

In [68]:
#creating a spark DataFrame for post_code data
Post_code_df =Raw_Post_code_data_split.toDF(['postcode','state','poa_code','area_sqkm','latitude','longitude','total_persons_males','total_persons_females','total_persons_persons','median_age_of_persons','median_total_personal_income_weekly','median_total_household_income_weekly','total_dwellings','geo_traffic_total_no_of_roads','geo_sum_of_commercial_traffic','geo_sum_of_non_commercial_traffic','geo_avg_commercial_traffic','geo_avg_non_commercial_traffic','geo_no_of_schools','geo_number_of_shopping_centre','geo_number_of_university','geo_populcation_growth_5yr','geo_nearest_weather_station_id','geo_distance_nearest_weather_meter'])
Post_code_df.take(1)

In [128]:
#Registering the post_code dataframe to a TempTable
Post_code_df.registerTempTable("Post_code_df")

In [125]:
#Calculating the weather condition (Sunny,Rain,Snow,Normal) according to the weather data

Final_data_set = sqlContext.sql("""
SELECT 
* , 
case when Rain_mm > 4 then 'Rain' 
    when  Temp_C < 1  then 'Snow'
    When  Temp_C > 25 and Rain_mm < 1 then 'Sunny'
    else 'Normal'
    end 
    as Weather_Condition
FROM 
Weather_data tab1  JOIN  Post_code_df tab2             
            ON tab1.PostCode = tab2.postcode
""")

##Registering the Final calculated Final_data_set to a TempTable
Final_data_set.registerTempTable("whole_data_set")

##whole_data.filter("Weather_Condition = 'Rain'").take(2)
Final_data_set.take(1)




[Row(Location=u'Little Bay', PostCode=u'2036', Temp_C=u'25', Dew_Point=u'12', Rel_Hum_%=u'79', Press_hPa=u'1019.3', Altitude_m=u'11', Rain_mm=u'0', Low_Temp_C=u'21.6', High_Temp_C=u'24', Latitude=u'33.9805 S', longtitude=u' 151.2430 E', Time_Stamp=u'2017-03-14 14:05:36', postcode=u'2036', state=u'NSW', poa_code=u'POA2036', area_sqkm=u'15.7135889234', latitude=u'-33.9708966144', longitude=u'151.2389567771', total_persons_males=u'13913', total_persons_females=u'12982', total_persons_persons=u'26895', median_age_of_persons=u'39', median_total_personal_income_weekly=u'563', median_total_household_income_weekly=u'1234', total_dwellings=u'9016', geo_traffic_total_no_of_roads=u'236', geo_sum_of_commercial_traffic=u'133340', geo_sum_of_non_commercial_traffic=u'1350260', geo_avg_commercial_traffic=u'565', geo_avg_non_commercial_traffic=u'5721', geo_no_of_schools=u'15', geo_number_of_shopping_centre=u'13', geo_number_of_university=u'0', geo_populcation_growth_5yr=u'0.266733', geo_nearest_weather

In [130]:
#Writing the calculated weather Data to a CSV file
df.write.format("com.databricks.spark.csv").option("header", "true").save('/home/ragavan/Downloads/Calculated_Weather_data.csv')

In [132]:
! pwd

/home/ragavan


In [85]:
#importing raw "ideal weather condition for each sport" file into spark RDD
game_spec_raw_file = sc.textFile("/home/ragavan/Downloads/sports.csv")

In [89]:
#Spliting the imported file with the delimiter
game_spec_split = game_spec_raw_file.map(lambda x:x.split(","))


In [88]:
#creating a spark DataFrame for the ideal weather condition for each sport
game_spec_df =game_spec_split.toDF(['game','high_temp','low_temp','max_rain_limit','max_dew_point','min_relative_humidity'])

[Row(game=u'cricket', high_temp=u'44', low_temp=u'20', max_rain_limit=u'5', max_dew_point=u'20', min_relative_humidity=u'80')]

In [92]:
#registering a temp table for future calculations
game_spec_df.registerTempTable("game_spec_df")

[Row(game=u'cricket', high_temp=u'44', low_temp=u'20', max_rain_limit=u'5', max_dew_point=u'20', min_relative_humidity=u'80')]